In [17]:
import csv
import cv2
import numpy as np
import h5py
import json
import numpy as np
import sklearn 

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers import MaxPooling2D
from keras.optimizers import Adam

relu = 'relu'

samples = []
with open('more_training_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
            
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
from sklearn.utils import shuffle
def generator(samples, batch_size=64):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            measurements = []
            for line in batch_samples:
                center_path = line[0]
                left_path = line[1]
                right_path = line[2]
#Adding center images
                filename_c = center_path.split('/')[-1]
                current_path_c = './more_training_data/IMG/' + filename_c
                image_c = cv2.imread(current_path_c)
                images.append(image_c)
#Adding left images
                filename_l = left_path.split('/')[-1]
                current_path_l = './more_training_data/IMG/' + filename_l
                image_l = cv2.imread(current_path_l)
                images.append(image_l)
#Adding right images
                filename_r = right_path.split('/')[-1]
                current_path_r = './more_training_data/IMG/' + filename_r
                image_r = cv2.imread(current_path_r)
                images.append(image_r)
#Adding steering angles from all of the three views
                correction=0.2
                steer = line[3]
                steer_left=float(steer)+float(correction)
                steer_right=float(steer)-float(correction)
                measurements.append(steer)
                measurements.append(steer_left)
                measurements.append(steer_right)
            augmented_images, augmented_measurements = [], []
            for image, measurement in zip(images, measurements):
                augmented_images.append(image)
                augmented_measurements.append(measurement)
                augmented_images.append(cv2.flip(image,1))
                augmented_measurements.append(float(measurement)*-1.0)

            X_train = np.array(augmented_images)
            Y_train = np.array(augmented_measurements)
            yield shuffle(X_train, Y_train)
            
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=4)
validation_generator = generator(validation_samples, batch_size=4)

#Build model
model = Sequential()
#Preprocess incoming data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(1,1))))

#Convolutional layers based on NVIDIA network

model.add(Convolution2D(24, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(36, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(48, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation(relu))
model.add(Dense(50))
model.add(Activation(relu))
model.add(Dense(10))
model.add(Activation(relu))
model.add(Dense(1))
model.summary()
model.compile(optimizer=Adam(0.0001), loss="mse")

model.fit_generator(train_generator, samples_per_epoch= len(samples), 
                    validation_data=validation_generator, 
                    nb_val_samples=len(samples)*0.5, nb_epoch=10)

model.save('model.h5', True)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_17 (Lambda)               (None, 160, 320, 3)   0           lambda_input_17[0][0]            
____________________________________________________________________________________________________
cropping2d_17 (Cropping2D)       (None, 65, 318, 3)    0           lambda_17[0][0]                  
____________________________________________________________________________________________________
convolution2d_81 (Convolution2D) (None, 33, 159, 24)   1824        cropping2d_17[0][0]              
____________________________________________________________________________________________________
activation_129 (Activation)      (None, 33, 159, 24)   0           convolution2d_81[0][0]           
___________________________________________________________________________________________

/home/sandorgoda/anaconda3/envs/carnd-term1-cpu/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


12432/12428 [==============================] - 235s - loss: 0.0216 - val_loss: 0.0176
Epoch 2/10
12432/12428 [==============================] - 239s - loss: 0.0160 - val_loss: 0.0152
Epoch 3/10
12432/12428 [==============================] - 252s - loss: 0.0151 - val_loss: 0.0157
Epoch 4/10
12432/12428 [==============================] - 236s - loss: 0.0145 - val_loss: 0.0127
Epoch 5/10
12444/12428 [==============================] - 239s - loss: 0.0143 - val_loss: 0.0156
Epoch 6/10
12432/12428 [==============================] - 229s - loss: 0.0146 - val_loss: 0.0128
Epoch 7/10
12432/12428 [==============================] - 238s - loss: 0.0133 - val_loss: 0.0149
Epoch 8/10
12432/12428 [==============================] - 238s - loss: 0.0128 - val_loss: 0.0134
Epoch 9/10
12432/12428 [==============================] - 235s - loss: 0.0133 - val_loss: 0.0137
Epoch 10/10
12444/12428 [==============================] - 239s - loss: 0.0126 - val_loss: 0.0136
